In [1]:
import python_utils as utils
import sys
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
from scipy.optimize import minimize, curve_fit
from scipy import stats
from statsmodels.stats.multitest import multipletests

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 13


# SIGMOIDPULSE0_002 t_mid = 13000 duration = 10 s

In [2]:
parseg = 250
overlap = 0.5

neglect_t = 3000
t_mid = 13000
t_mid2 = 23000
sigm_par = 0.002

t_tot = 33000

dd_low = 0.850

function = "sigmoidpulse"

In [3]:
sigm_par_poiss = "{:.5f}".format(sigm_par)

Dd = "0.950"
subnets = "STN"
simulations = [str(i) for i in range(32)]


In [4]:
Dd_values = []
current_value = dd_low
while current_value <= float(Dd):
    Dd_values.append(round(current_value, 4))
    current_value += 0.0025

print(Dd_values)

[0.85, 0.8525, 0.855, 0.8575, 0.86, 0.8625, 0.865, 0.8675, 0.87, 0.8725, 0.875, 0.8775, 0.88, 0.8825, 0.885, 0.8875, 0.89, 0.8925, 0.895, 0.8975, 0.9, 0.9025, 0.905, 0.9075, 0.91, 0.9125, 0.915, 0.9175, 0.92, 0.9225, 0.925, 0.9275, 0.93, 0.9325, 0.935, 0.9375, 0.94, 0.9425, 0.945, 0.9475, 0.95]


In [5]:
simulations_folder_path = f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/"
print(simulations_folder_path)

./build/output/n1/sigmoidpulse_13000.00_0.00200_0.00_0.950_1.00_1.00_1.6/


# FLAT. media per ogni valore di dd

In [ ]:
function1 = "flat"
d = ["{:.4f}".format(elemento) for elemento in Dd_values]
simulations = [str(i) for i in range(32)]
pow_t_mean_all = []
for simulation in simulations:

    pow_t_mean = []
    t = []
    f = []
    print(simulation)

    for ddd in d:
        print(ddd)
        periodogram_path = './periodograms/' + ddd.replace('.','_') + '_' + subnets + '_' + function1 + '_' + simulation
        if not os.path.exists(periodogram_path):  #check if periodogram folder exists and if not create one
            os.makedirs(periodogram_path)

        data = np.loadtxt("./build/output/n1/" + function1 + "_0.00_0.00000_0.00_" + ddd + "_1.00_1.00_1.6/" + simulations[0] + "/ext_rateD2.txt").T
        data[0] = data[0]/1000
        data[1] = data[1]/1.083

        s = utils.SpikeSim("./build/output/n1/" + function1 + "_0.00_0.00000_0.00_" + ddd + "_1.00_1.00_1.6/" + simulation , 'new_sim_parallel.yaml', neglect_t, np.max(data[0]))
        output=s.periodogram(pop=subnets, res=1., N_parseg=parseg, save_img='')

        f = output[0]
        t = output[1]
        Sxx = output[2]

        mask = (8 <= f) & (f <= 26)
        Sxx_lim = Sxx[mask,:]
        f_lim = f[mask]

        pow_t_mean.append(np.mean(Sxx_lim))

    ###################################################################################################################

    pow_t_mean_all.append(pow_t_mean)
print(pow_t_mean_all)

#np.savetxt('./build/output/n1/new_pow_t_mean' + str(parseg) + '.txt', pow_t_mean_all, header='pow_t_mean', delimiter='\t')

0
0.8500
nparseg = 250	noverlap=125
0.8525


## funzione dell'intensità della sigmoide attesa

In [6]:
pow_t_mean_all = np.loadtxt('./build/output/n1/new_pow_t_mean' + str(parseg) + '.txt', delimiter='\t', skiprows=1)
x = pow_t_mean_all[:]
pow_t_mean_point_int= [elemento[-len(Dd_values):] for elemento in x]
for i in range(32):
    pow_t_mean_point_int[i] = pow_t_mean_point_int[i].tolist()
    pow_t_mean_point_int[i] = pow_t_mean_point_int[i][::-1] + pow_t_mean_point_int[i]
    pow_t_mean_point_int[i] = np.insert(pow_t_mean_point_int[i], 0, 0)
    pow_t_mean_point_int[i] = np.array(pow_t_mean_point_int[i])
pow_t_mean_point_int = np.array(pow_t_mean_point_int)



In [7]:
Dd_values_tot = Dd_values[::-1] + Dd_values
intermediate_points = [round((Dd_values_tot[i] + Dd_values_tot[i + 1]) / 2, 4) for i in range(len(Dd_values_tot) - 1)]


In [8]:
# Inizializza variabile di controllo per il secondo loop
start_second_loop = False

t_d_int = []
t_d_int.append(t_tot)
# Primo loop
for x in intermediate_points[:]:
    # Controlla se è il momento di iniziare il secondo loop
    if x == dd_low:
        start_second_loop = True
        t_d_int.append((t_mid2 + t_mid)/2)
        continue

    # Calcola i valori in base alla variabile di controllo
    if not start_second_loop:
        y = -np.log((0.95 - dd_low) / (x - dd_low) - 1) / sigm_par + t_mid2
    else:
        y = -np.log((dd_low - 0.95) / (x - 0.95) - 1) / sigm_par + t_mid

    # Aggiungi il valore calcolato a t_d_int
    t_d_int.append(y)

t_d_int.append(0)

t_d_int = np.array(t_d_int)
t_d_int = np.round(t_d_int).astype(int)


In [10]:
%matplotlib widget
def genera_coppie_tempo_intensita(tempo_array, intensita_array):
    y = []
    
    for tempo in range(tempo_array[-1]):
        if np.isin(tempo, tempo_array):
            indices = np.where(tempo_array == tempo)[0]
            y.append(intensita_array[:][indices[0]])
        else:
            y.append(y[-1])  # Ripeti l'ultimo valore di intensita_array se tempo non è presente
    
    return np.array(y)

power = []
for i in range(32):
    power_i = genera_coppie_tempo_intensita(t_d_int[::-1], pow_t_mean_point_int[i][::-1])
    power.append(power_i)

    # Plot dei dati
    plt.plot( power_i, marker='o')
    plt.xlabel('Time [ms]')
    plt.ylabel('Mean β PSD')
    #plt.title('Andamento Tempo-Intensità')
    plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"power_{cleaned_path}_{parseg}.txt"
# Costruisci il percorso completo del file. Assicurati che questo percorso punti alla cartella desiderata
file_path = os.path.join(simulations_folder_path, filename)

# Salva i dati in un file txt
np.savetxt(file_path, power, fmt='%f', header="power")


In [6]:
####################### DATI SIMULAZIONE #######################################
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"power_{cleaned_path}_{parseg}.txt"

power = np.loadtxt(simulations_folder_path + filename, skiprows=1)


In [7]:
time = np.arange(0, len(power[0]), 1)
df_attesi = []

for i in range(32):
    # Crea un DataFrame con i dati
    df_i = pd.DataFrame({'Tempo': time, 'Valore': power[i]})

    # Calcola la media mobile e assegna i risultati al tempo centrale
    window_size = parseg
    df_i['Media_Mobile'] = df_i['Valore'].rolling(window=window_size, center=True).mean()

    # Rimuovi le righe con valori mancanti (prime e ultime finestre)
    df_i = df_i.dropna()

    # Rimuovi i primi neglect_t elementi
    df_i = df_i.iloc[neglect_t:]

    # Aggiungi il DataFrame alla lista
    df_attesi.append(df_i)

# Stampa il DataFrame risultante
#print(df_attesi)

In [8]:
media = []
error = []
tt = []

for i in range(0, len(df_attesi[0]['Tempo']), int(parseg - parseg*overlap)): # qui metti 0. overlap
    media.append(np.mean([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    error.append(np.std([df_i['Media_Mobile'].iloc[i] for df_i in df_attesi]))
    tt.append(i + int(parseg*overlap))


In [9]:
%matplotlib widget
tt = np.array(tt)/1000
plt.figure(figsize=(12, 6))
plt.errorbar(tt, media, yerr=error, fmt='o')
plt.xlabel('Time [s]')
plt.ylabel('mean β PSD [u.a.]')
plt.title('Pulse expected mean β PSD')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# simulazione SIGMOIDPULSE

In [ ]:
%matplotlib inline
#funzione Dd
data = np.loadtxt(simulations_folder_path + simulations[0] + "/ext_rateD2.txt").T

data[0] = data[0]/1000
data[1] = data[1]/1.083

#check if periodogram folder exists and if not create one
#spikesim
periodogram_paths = []
s = []
for simulation in simulations:
    periodogram_path = './periodograms/' + Dd.replace('.','_') + '_' + subnets + '_' + function + '_' + simulation
    if not os.path.exists(periodogram_path):  #check if periodogram folder exists and if not create one
        os.makedirs(periodogram_path) 
    periodogram_paths.append(periodogram_path)
    ss = utils.SpikeSim(simulations_folder_path + simulation , 'new_sim_parallel.yaml', neglect_t, np.max(data[0]))
    s.append(ss)

all_pow_t = []
t = []
f = []

for index,ss in enumerate(s):
    output=ss.periodogram(pop=subnets, res=1., N_parseg=parseg, save_img=periodogram_paths[index] + "/periodogram.png")

    f = output[0]
    t = output[1]
    Sxx = output[2]

    mask = (8 <= f) & (f <= 26)
    Sxx_lim = Sxx[mask,:]
    f_lim = f[mask]

    pow_t = []
    for i in range (len(t)):
        pow_t.append(np.mean(Sxx_lim[:,i]))

    all_pow_t.append(pow_t)

np.vstack(all_pow_t)
pow_t_mean = np.mean(all_pow_t, axis=0)
pow_error = np.std(all_pow_t, axis=0)

In [ ]:
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"all_pow_t{cleaned_path}_{parseg}.txt"
# Costruisci il percorso completo del file. Assicurati che questo percorso punti alla cartella desiderata
file_path = os.path.join(simulations_folder_path, filename)

# Salva i dati in un file txt
np.savetxt(file_path, all_pow_t, fmt='%f', header="t\pow_t_mean\pow_error")


In [10]:
####################### DATI SIMULAZIONE #######################################
cleaned_path = simulations_folder_path.replace("./build/output/n1/", "").replace("_1.00_1.00_1.6/", "")
filename = f"all_pow_t{cleaned_path}_{parseg}.txt"

all_pow_t = np.loadtxt(simulations_folder_path + filename, skiprows=1)


In [11]:
pow_t_mean = np.mean(all_pow_t, axis=0)
pow_error = np.std(all_pow_t, axis=0)

num_elements = len(pow_t_mean)
t = np.linspace(parseg*overlap/1000, num_elements * parseg*overlap/1000, num_elements)


## time-pow_t

In [12]:
%matplotlib widget
# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(9, 5))

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(t, pow_t_mean, color='orange')

# Etichette e legenda per l'asse x e y
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Mean β PSD [u.a.]')

plt.title(f'Mean β PSD for τ = {1/sigm_par/1000} [s]')# nparseg = {parseg}')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
%matplotlib widget
# Creare un grafico dei valori nel tempo
fig, ax1 = plt.subplots(figsize=(9, 5))

# Utilizza la funzione errorbar per tracciare i dati con errori associati
ax1.errorbar(t, pow_t_mean, yerr=pow_error, fmt='none', capsize=1, color='black', ecolor='black', elinewidth=1)

# Plot dei dati originali
ax1.plot(t, pow_t_mean, label='Dynamic simulation β PSD', color='orange')
#ax1.set_yscale('log')

# Aggiungi il terzo set di dati con errori associati al grafico esistente
plt.errorbar(tt, media, yerr=error, fmt='o', label='Expected PSD')

# Etichette e legenda per l'asse x e y
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Mean β PSD [u.a.]')
ax1.legend()  # Assicurati di avere tutte le etichette delle leggende necessarie

plt.title(f'Intensità nel Range β for τ = {1/sigm_par/1000} [s]')# nparseg = {parseg}')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# visualizza dopamina

In [14]:
%matplotlib widget
data_path = f"./build/output/n1/{function}_{t_mid}.00_{sigm_par_poiss}_0.00_{Dd}_1.00_1.00_1.6/0/ext_rateD2.txt"
data = np.loadtxt(data_path).T
data[0] = data[0] / 1000  # Converti il tempo in secondi
data[1] = 10*(data[1] / 1.083 - 0.85)

data = data[:, neglect_t:]

# Crea un grafico per ogni set di dati
plt.figure(figsize=(12, 5))
plt.plot(data[0], data[1], color ='red')
plt.xlabel('Time (s)')
plt.ylabel('Dopamine Depletion y [a.u.]')
plt.title('Dopamine Concentration Over Time')
plt.ylim(-0.1, 1.1) 
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …